In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10134,2024-04-30,Romênia Divisão A,13:00,Arges Pitesti,Cluj-Napoca,5.03,1.15,159.5,1.83,1.85,0.0,0.00,0.00,GEqVCDt2,0.198807,0.869565,0.546448,0.540541,0.068372,395.526,280.933303,0.710278,1.8,1.643168,0.912871,680.80,5.328,4.003407,0.751390,-21.0,96.632,10.603500,0.109731,3.0,0.000000,0.000000,105.04,1.056,0.042190,0.039953,129.0,74,101,9.20,1.04,173.894,112.688,0.887888,0.007686,NaN,-0.65,-0.130,-31.000000,0.000000,0.0,0.000000,1.06,0.212,0.707547,0.856742,1.0,0.143258
10135,2024-04-30,Romênia Divisão A,12:00,Laguna Bucuresti,Petrolul Ploiesti,4.51,1.18,152.5,1.84,1.86,0.0,0.00,0.00,rk3CPjQ7,0.221729,0.847458,0.543478,0.537634,0.069187,569.464,174.567431,0.306547,0.0,0.000000,NaN,747.60,7.682,2.042785,0.265918,-60.0,136.764,26.463719,0.193499,1.2,1.643168,1.369306,96.72,1.788,0.532607,0.297879,-1.0,84,93,8.90,1.04,419.024,152.612,0.827650,0.007644,NaN,2.28,0.456,7.697368,0.000000,0.0,0.000000,-0.88,-0.176,-1.022727,0.489269,0.4,-0.089269
10136,2024-04-30,Romênia Divisão A,11:00,CSM Focsani,CSM Targu Jiu,1.83,1.88,167.5,1.83,1.85,-1.5,1.96,2.05,jeQwtCAl,0.546448,0.531915,0.546448,0.540541,0.078363,287.952,133.832813,0.464775,1.2,1.643168,1.369306,437.80,3.358,0.968101,0.288297,-14.0,125.640,30.694727,0.244307,0.6,1.341641,2.236068,104.92,1.550,0.470478,0.303535,-48.0,110,86,3.98,1.22,164.234,224.656,0.019059,0.007686,0.031740,3.80,0.760,1.092105,0.477395,0.4,-0.077395,1.52,0.304,2.894737,0.329892,0.3,-0.029892
10137,2024-04-30,Macedônia Do Norte Prva Liga,14:00,MZT Skopje,TFT Skopje,1.14,5.24,148.5,1.80,1.88,0.0,0.00,0.00,KQdscApm,0.877193,0.190840,0.555556,0.531915,0.068033,98.702,9.251955,0.093736,3.0,0.000000,0.000000,109.44,1.066,0.045056,0.042266,113.0,129.108,35.007253,0.271147,1.8,1.643168,0.912871,96.60,1.616,0.418665,0.259075,18.0,96,70,1.14,1.38,92.354,157.760,0.908821,0.030744,NaN,0.50,0.100,1.400000,0.879472,0.8,-0.079472,-0.20,-0.040,-106.000000,0.515668,0.5,-0.015668
10138,2024-04-30,Israel Liga Leumit,13:00,Maccabi Haifa,Ramat Hasharon,1.45,2.62,163.5,1.84,1.86,0.0,0.00,0.00,Gv5coUFj,0.689655,0.381679,0.543478,0.537634,0.071335,212.130,85.034562,0.400861,1.2,1.643168,1.369306,268.80,2.514,0.790873,0.314587,1.0,161.394,30.577769,0.189460,0.6,1.341641,2.236068,194.56,2.478,0.659485,0.266136,-84.0,105,76,2.56,2.56,188.072,193.714,0.406543,0.007644,NaN,1.87,0.374,1.203209,0.476615,0.5,0.023385,-5.00,-1.000,-1.620000,0.390846,0.1,-0.290846
10139,2024-04-30,Islândia Premier League,16:15,Grindavik,Keflavik,1.37,2.91,186.5,1.80,1.86,0.0,0.00,0.00,fuLIHKcd,0.729927,0.343643,0.555556,0.537634,0.073570,155.348,31.305526,0.201519,2.4,1.341641,0.559017,122.85,1.590,0.303562,0.190920,66.0,158.242,56.731795,0.358513,1.8,1.643168,0.912871,235.98,1.720,0.338378,0.196732,13.0,91,114,1.35,2.07,165.328,178.236,0.508853,0.023184,NaN,3.27,0.654,0.565749,0.640499,0.8,0.159501,0.24,0.048,39.791667,0.545551,0.6,0.054449
10140,2024-04-30,Israel Superliga,12:00,Hapoel Jerusalem,Maccabi Ironi Ramat Gan,1.34,3.14,163.5,1.87,1.86,0.0,0.00,0.00,2o4IFlPo,0.746269,0.318471,0.534759,0.537634,0.064740,148.550,73.938267,0.497733,1.8,1.643168,0.912871,92.00,1.930,0.957940,0.496342,-22.0,215.782,129.454642,0.599933,3.0,0.000000,0.000000,440.86,2.414,1.368313,0.566824,41.0,80,94,1.15,4.69,106.388,269.726,0.568211,0.003791,NaN,-1.35,-0.270,-1.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10150,19:15,Uruguai Liga Uruguaia,Defensor Sporting,Hebraica y Macabi,1.70,Back Home
10177,21:30,Argentina Liga A,Obera TC,Olimpico,2.12,Back Home
10189,15:00,França Lnb,Gravelines-Dunkerque,JL Bourg,2.47,Back Home
